In [1]:
import pandas as pd
import os
from rapidfuzz import fuzz
import re
from unidecode import unidecode

In [2]:
player_stats_df = pd.read_csv('./data/Player_stats/all_players.csv')
keeper_stats_df = pd.read_csv('./data/Player_stats/all_goalkeepers.csv')

In [3]:
keeper_stats_df

,Player,Club,POS,GP,GS,MINS,SHTS,SV,GA,GAA,PKG/A,W,L,T,ShO,W%,Sv%,Year,Season
0,Mark Dodd,DAL,GK,31,31,2776,235,160,45,1.45,5/5,17,14,0,6,54.8,68.1,1996,reg
1,Tony Meola,MET,GK,29,29,2610,188,142,38,1.31,3/4,14,15,0,9,48.3,75.5,1996,reg
2,Mark Dougherty,TB,GK,28,28,2520,186,123,47,1.68,9/10,17,11,0,3,60.7,66.1,1996,reg
3,Jorge Campos,LA,GK,24,24,2160,133,100,27,1.13,0/1,13,8,0,4,54.2,75.2,1996,reg
4,Garth Lagerwey,KC,GK,23,22,1959,133,83,38,1.73,2/2,12,10,0,3,52.2,62.4,1996,reg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001,Adrian Zendejas,NaN,GK,0,0,0,0,0,0,0.00,0/0,0,0,0,0,0.0,0.0,2020,post
2002,Mason Stajduhar,NaN,GK,0,0,0,0,0,0,0.00,0/0,0,0,0,0,0.0,0.0,2020,post
2003,Brady Scott,NaN,GK,0,0,0,0,0,0,0.00,0/0,0,0,0,0,0.0,0.0,2020,post
2004,Kendall McIntosh,NaN,GK,0,0,0,0,0,0,0.00,0/0,0,0,0,0,0.0,0.0,2020,post


In [4]:
player_stats_df

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,...,RdA,A/90min,FC,FS,OFF,YC,RC,SOG%,Year,Season
0,Roy Lassiter,TB,F,30,30,2580,27,4,76,49,...,2,0.14,20,39,70,2,0,64.47,1996,reg
1,Raul Diaz Arce,DC,F,28,28,2351,23,2,100,49,...,2,0.08,32,26,35,6,1,49.00,1996,reg
2,Eduardo Hurtado,LA,F,26,26,2323,21,7,87,56,...,3,0.27,48,26,25,5,0,64.37,1996,reg
3,Preki,KC,M,32,32,2880,18,13,140,61,...,4,0.41,26,44,7,3,0,43.57,1996,reg
4,Brian McBride,CLB,F,28,28,2307,17,3,79,44,...,1,0.12,21,46,10,0,0,55.70,1996,reg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15762,Alejandro Pozuelo,NaN,M,1,1,120,0,0,5,0,...,0,0.00,2,0,0,0,0,0.00,2020,post
15763,Franco Jara,NaN,F,2,2,210,0,0,5,1,...,0,0.00,6,2,3,0,0,20.00,2020,post
15764,Andy Polo,NaN,M,1,1,105,0,0,6,3,...,0,0.00,0,3,0,0,0,50.00,2020,post
15765,Cristian Roldan,NaN,M,4,4,360,0,0,6,1,...,0,0.00,5,11,0,0,0,16.67,2020,post


In [5]:
player_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15767 entries, 0 to 15766
Data columns (total 28 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Player   15767 non-null  object 
 1   Club     10274 non-null  object 
 2   POS      15767 non-null  object 
 3   GP       15767 non-null  int64  
 4   GS       15767 non-null  int64  
 5   MINS     15767 non-null  int64  
 6   G        15767 non-null  int64  
 7   A        15767 non-null  int64  
 8   SHTS     15767 non-null  int64  
 9   SOG      15767 non-null  int64  
 10  GWG      15767 non-null  int64  
 11  PKG/A    15767 non-null  object 
 12  HmG      15767 non-null  int64  
 13  RdG      15767 non-null  int64  
 14  G/90min  15767 non-null  float64
 15  SC%      15767 non-null  float64
 16  GWA      15767 non-null  int64  
 17  HmA      15767 non-null  int64  
 18  RdA      15767 non-null  int64  
 19  A/90min  15767 non-null  float64
 20  FC       15767 non-null  int64  
 21  FS       157

In [6]:
player_stats_df['POS'].isna().sum()

0

In [7]:
keeper_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2006 entries, 0 to 2005
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  2006 non-null   object 
 1   Club    1324 non-null   object 
 2   POS     2006 non-null   object 
 3   GP      2006 non-null   int64  
 4   GS      2006 non-null   int64  
 5   MINS    2006 non-null   int64  
 6   SHTS    2006 non-null   int64  
 7   SV      2006 non-null   int64  
 8   GA      2006 non-null   int64  
 9   GAA     2006 non-null   float64
 10  PKG/A   2006 non-null   object 
 11  W       2006 non-null   int64  
 12  L       2006 non-null   int64  
 13  T       2006 non-null   int64  
 14  ShO     2006 non-null   int64  
 15  W%      2006 non-null   float64
 16  Sv%     2006 non-null   float64
 17  Year    2006 non-null   int64  
 18  Season  2006 non-null   object 
dtypes: float64(3), int64(11), object(5)
memory usage: 297.9+ KB


In [8]:
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

In [9]:
player_stats_df = player_stats_df[player_stats_df['Year'].isin(years)]
keeper_stats_df = keeper_stats_df[keeper_stats_df['Year'].isin(years)]

In [10]:
player_stats_df

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,...,RdA,A/90min,FC,FS,OFF,YC,RC,SOG%,Year,Season
5067,Chris Wondolowski,SJ,F,32,31,2813,27,7,127,55,...,3,0.22,19,25,28,3,0,43.31,2012,reg
5068,Kenny Cooper,NY,F,33,26,2505,18,3,84,46,...,0,0.11,36,28,20,3,0,54.76,2012,reg
5069,Alvaro Saborio,RSL,F,31,26,2390,17,3,87,40,...,2,0.11,35,37,37,5,0,45.98,2012,reg
5070,Robbie Keane,LA,F,28,28,2519,16,9,94,43,...,3,0.32,15,34,27,0,0,45.74,2012,reg
5071,Thierry Henry,NY,F,25,24,2108,15,12,88,37,...,3,0.51,18,26,24,5,0,42.05,2012,reg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15762,Alejandro Pozuelo,NaN,M,1,1,120,0,0,5,0,...,0,0.00,2,0,0,0,0,0.00,2020,post
15763,Franco Jara,NaN,F,2,2,210,0,0,5,1,...,0,0.00,6,2,3,0,0,20.00,2020,post
15764,Andy Polo,NaN,M,1,1,105,0,0,6,3,...,0,0.00,0,3,0,0,0,50.00,2020,post
15765,Cristian Roldan,NaN,M,4,4,360,0,0,6,1,...,0,0.00,5,11,0,0,0,16.67,2020,post


In [11]:
player_stats_df['Season'].value_counts()

Season
reg     5207
post    2687
Name: count, dtype: int64

In [12]:
player_stats_df = player_stats_df[player_stats_df['Season'] == 'reg']
keeper_stats_df = keeper_stats_df[keeper_stats_df['Season'] == 'reg']

In [13]:
player_stats_df

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,...,RdA,A/90min,FC,FS,OFF,YC,RC,SOG%,Year,Season
5067,Chris Wondolowski,SJ,F,32,31,2813,27,7,127,55,...,3,0.22,19,25,28,3,0,43.31,2012,reg
5068,Kenny Cooper,NY,F,33,26,2505,18,3,84,46,...,0,0.11,36,28,20,3,0,54.76,2012,reg
5069,Alvaro Saborio,RSL,F,31,26,2390,17,3,87,40,...,2,0.11,35,37,37,5,0,45.98,2012,reg
5070,Robbie Keane,LA,F,28,28,2519,16,9,94,43,...,3,0.32,15,34,27,0,0,45.74,2012,reg
5071,Thierry Henry,NY,F,25,24,2108,15,12,88,37,...,3,0.51,18,26,24,5,0,42.05,2012,reg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15296,Sebastian Mendez,ORL,M,19,11,1087,0,0,17,3,...,0,0.00,32,9,1,4,1,17.65,2020,reg
15297,Kelyn Rowe,NE,M,16,10,896,0,0,17,4,...,0,0.00,13,4,0,3,0,23.53,2020,reg
15298,Siem de Jong,CIN,M,15,8,793,0,0,18,5,...,0,0.00,7,8,1,0,0,27.78,2020,reg
15299,Maxime Chanot,NYC,D,20,20,1712,0,0,19,4,...,0,0.00,18,6,1,5,1,21.05,2020,reg


In [14]:
keeper_stats_df

,Player,Club,POS,GP,GS,MINS,SHTS,SV,GA,GAA,PKG/A,W,L,T,ShO,W%,Sv%,Year,Season
674,Jimmy Nielsen,KC,GK,34,34,3060,102,77,27,0.79,3/4,18,7,9,15,52.9,75.5,2012,reg
675,Donovan Ricketts,MTL,GK,33,33,2913,143,87,52,1.58,8/8,12,14,5,5,36.4,60.8,2012,reg
676,Andy Gruenebaum,CLB,GK,33,33,2920,166,124,41,1.24,3/4,13,12,7,8,39.4,74.7,2012,reg
677,Tally Hall,HOU,GK,33,33,2946,129,88,39,1.18,5/5,14,8,11,12,42.4,68.2,2012,reg
678,Jon Busch,SJ,GK,32,32,2835,123,77,40,1.25,4/5,17,5,9,7,53.1,62.6,2012,reg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1940,Emmanuel Ochoa,SJ,GK,0,0,0,0,0,0,0.00,0/0,0,0,0,0,0.0,0.0,2020,reg
1941,Drake Callender,MIA,GK,0,0,0,0,0,0,0.00,0/0,0,0,0,0,0.0,0.0,2020,reg
1942,Elliot Panicco,NSH,GK,0,0,0,0,0,0,0.00,0/0,0,0,0,0,0.0,0.0,2020,reg
1943,Eric Dick,SKC,GK,0,0,0,0,0,0,0.00,0/0,0,0,0,0,0.0,0.0,2020,reg


In [15]:
salary2012_df = pd.read_excel('./data/Salary_data/2012-Salaries.xlsx', header=1)
salary2013_df = pd.read_excel('./data/Salary_data/2013-Salaries.xlsx', header=1)
salary2014_df = pd.read_excel('./data/Salary_data/2014-Salaries.xlsx', header=1)
salary2015_df = pd.read_excel('./data/Salary_data/2015-Salaries.xlsx', header=1)
salary2016_df = pd.read_excel('./data/Salary_data/2016-Salaries.xlsx', header=1)
salary2017_df = pd.read_excel('./data/Salary_data/2017-Salaries.xlsx', header=1)
salary2018_df = pd.read_excel('./data/Salary_data/2018-Salaries.xlsx', header=0)
salary2019_df = pd.read_excel('./data/Salary_data/2019-Salaries.xlsx', header=1)
salary2020_df = pd.read_excel('./data/Salary_data/2020-Salaries-.xlsx', header=0)


In [16]:
salary2020_df

,Last Name,First Name,2020 Fall Salary List Club,Playing Position,Base Salary,Unnamed: 5,Base Guaranteed Comp.
0,Aaronson,Brenden,Philadelphia Union,M,85000,NaN,103309
1,Abdul-Salaam,Saad,FC Cincinnati,D,81375,NaN,90042
2,Abu,Mohammed,DC United,M,81375,NaN,81375
3,Abubakar,Lalas,Colorado Rapids,D,145000,NaN,154937
4,Accam,David,Nashville SC,F,875000,NaN,1002916
...,...,...,...,...,...,...,...
795,NaN,Kaku,New York Red Bulls,M,850000,NaN,850000
796,NaN,Robinho,Orlando City SC,M,198000,NaN,205500
797,NaN,Ilsinho,Philadelphia Union,M,425000,NaN,476250
798,NaN,Judson,San Jose Earthquakes,M,320000,NaN,325000


In [17]:
salary2020_df.head()

,Last Name,First Name,2020 Fall Salary List Club,Playing Position,Base Salary,Unnamed: 5,Base Guaranteed Comp.
0,Aaronson,Brenden,Philadelphia Union,M,85000,NaN,103309
1,Abdul-Salaam,Saad,FC Cincinnati,D,81375,NaN,90042
2,Abu,Mohammed,DC United,M,81375,NaN,81375
3,Abubakar,Lalas,Colorado Rapids,D,145000,NaN,154937
4,Accam,David,Nashville SC,F,875000,NaN,1002916


In [18]:
salary2018_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705 entries, 0 to 704
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Club           696 non-null    object
 1   Last Name      705 non-null    object
 2   First Name     704 non-null    object
 3   Pos            705 non-null    object
 4   Salary         705 non-null    object
 5   Compensation   705 non-null    object
dtypes: object(6)
memory usage: 33.2+ KB


In [19]:
salary_dict = {"2012": salary2012_df, "2013": salary2013_df, "2014": salary2014_df, "2015": salary2015_df, 
               "2016": salary2016_df, "2017": salary2017_df, "2018": salary2018_df, "2019": salary2019_df, "2020": salary2020_df}

In [20]:
for year in salary_dict.keys():
    salary_dict[year].columns = salary_dict[year].columns.str.strip()
   #salary = salary[salary.map(lambda x: " " not in str(x)).all(axis=1)]

In [21]:
salary_dict['2012'].columns = salary_dict['2012'].columns.str.strip()

In [22]:
salary_dict['2012'].head()

,Club,Last Name,First Name,Pos,Base Salary,Compensation
0,PHI,Adu,Freddy,M,400000,519000
1,CHV,Agudelo,Juan,F,70000,100000
2,KC,Aiyegbusi,Olukorede,D,44100,44100
3,COL,Akpan,Andre,F,57200,57200
4,PHI,Albright,Chris,D,55000,55000


In [23]:
salary2018_df.rename(columns={'Salary': 'Base Salary'}, inplace=True)
salary2018_df

,Club,Last Name,First Name,Pos,Base Salary,Compensation
0,New York Red Bulls,Abang,Anatole,F,68927,68927
1,New York City FC,Abdul-Salaam,Saad,D,106480,120230
2,Columbus Crew,Abu,Mohammed,M,175008,181258
3,Columbus Crew,Abubakar,Lalas,D,68250.04,75750.04
4,Philadelphia Union,Accam,David,F-M,1250000.04,1250000.04
...,...,...,...,...,...,...
700,FC Dallas,Ziegler,Reto,D,750000,800000
701,LAFC,Zimmerman,Walker,D,235000,235000
702,Atlanta United,Zizzo,Sal,D,129999.96,129999.96
703,FC Dallas,Zobeck,Kyle,GK,67500,67500


In [24]:
salary_dict["2018"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705 entries, 0 to 704
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Club          696 non-null    object
 1   Last Name     705 non-null    object
 2   First Name    704 non-null    object
 3   Pos           705 non-null    object
 4   Base Salary   705 non-null    object
 5   Compensation  705 non-null    object
dtypes: object(6)
memory usage: 33.2+ KB


In [25]:
salary2019_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714 entries, 0 to 713
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Club                         714 non-null    object 
 1   Last Name                    714 non-null    object 
 2   First Name                   705 non-null    object 
 3   Playing Position             713 non-null    object 
 4   CY Salary (Annual)           714 non-null    float64
 5   CY Guaranteed Comp (Annual)  714 non-null    float64
dtypes: float64(2), object(4)
memory usage: 33.6+ KB


In [26]:
salary2019_df.rename(columns={'Playing Position': 'Pos', 'CY Salary (Annual)': 'Base Salary',
                              'CY Guaranteed Comp (Annual)': 'Compensation'}, inplace=True)

In [27]:
salary2020_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Last Name                   795 non-null    object 
 1   First Name                  800 non-null    object 
 2   2020 Fall Salary List Club  800 non-null    object 
 3   Playing Position            798 non-null    object 
 4   Base Salary                 765 non-null    object 
 5   Unnamed: 5                  35 non-null     float64
 6   Base Guaranteed Comp.       800 non-null    object 
dtypes: float64(1), object(6)
memory usage: 43.9+ KB


In [28]:
salary2020_df.rename(columns={'Playing Position': 'Pos', '2020 Fall Salary List Club': 'Club'}, inplace=True)

In [29]:
df_year = 2012
for year in salary_dict.keys():
    salary_dict.get(year, {})['year'] = df_year
    df_year+=1
    

In [30]:
salary2015_df.head()

,Club,Last Name,First Name,Pos,Base Salary,Compensation,year
0,NY,Abang,Anatole,F,50000,50000,2015
1,KC,Abdul-Salaam,Saad,D,60000,73750,2015
2,CHI,Accam,David,F,650000,720937.5,2015
3,DAL,Acosta,Kellyn,M,60000,84000,2015
4,VAN,Adekugbe,Samuel,D,60000,65000,2015


In [31]:
salary2020_df['Base Salary'].fillna(salary2020_df['Unnamed: 5'], inplace=True)
salary2020_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Last Name              795 non-null    object 
 1   First Name             800 non-null    object 
 2   Club                   800 non-null    object 
 3   Pos                    798 non-null    object 
 4   Base Salary            800 non-null    object 
 5   Unnamed: 5             35 non-null     float64
 6   Base Guaranteed Comp.  800 non-null    object 
 7   year                   800 non-null    int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 50.1+ KB


In [32]:
salary2020_df = pd.DataFrame(salary2020_df, columns=['Club', 'Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'year'])
salary2020_df

,Club,Last Name,First Name,Pos,Base Salary,Base Guaranteed Comp.,year
0,Philadelphia Union,Aaronson,Brenden,M,85000,103309,2020
1,FC Cincinnati,Abdul-Salaam,Saad,D,81375,90042,2020
2,DC United,Abu,Mohammed,M,81375,81375,2020
3,Colorado Rapids,Abubakar,Lalas,D,145000,154937,2020
4,Nashville SC,Accam,David,F,875000,1002916,2020
...,...,...,...,...,...,...,...
795,New York Red Bulls,NaN,Kaku,M,850000,850000,2020
796,Orlando City SC,NaN,Robinho,M,198000,205500,2020
797,Philadelphia Union,NaN,Ilsinho,M,425000,476250,2020
798,San Jose Earthquakes,NaN,Judson,M,320000,325000,2020


In [33]:
salary2020_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Club                   800 non-null    object
 1   Last Name              795 non-null    object
 2   First Name             800 non-null    object
 3   Pos                    798 non-null    object
 4   Base Salary            800 non-null    object
 5   Base Guaranteed Comp.  800 non-null    object
 6   year                   800 non-null    int64 
dtypes: int64(1), object(6)
memory usage: 43.9+ KB


In [34]:
salary2012_df['Pos'].value_counts()

Pos
M      188
D      141
F      126
GK      64
M-F     11
Pos     10
M-D      8
D-M      5
M/F      4
D/M      2
F-M      2
MF       1
M/D      1
D/F      1
Name: count, dtype: int64

In [35]:
salary2012_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574 entries, 0 to 573
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Club          562 non-null    object
 1   Last Name     564 non-null    object
 2   First Name    562 non-null    object
 3   Pos           564 non-null    object
 4   Base Salary   574 non-null    object
 5   Compensation  574 non-null    object
 6   year          574 non-null    int64 
dtypes: int64(1), object(6)
memory usage: 31.5+ KB


In [36]:
for year, df in salary_dict.items():
    # Remove columns i dont need
    salary_dict[year] = salary_dict[year].drop(['Club'], axis=1)

# Print the modified DataFrames
for df in salary_dict.values():
    print(df)

     Last Name First Name  Pos Base Salary Compensation  year
0          Adu     Freddy    M      400000       519000  2012
1      Agudelo       Juan    F       70000       100000  2012
2    Aiyegbusi  Olukorede    D       44100        44100  2012
3        Akpan      Andre    F       57200        57200  2012
4     Albright      Chris    D       55000        55000  2012
..         ...        ...  ...         ...          ...   ...
569     Zapata       Luis    D       65460     69043.33  2012
570     Zayner        Jed    D       60000     61666.67  2012
571   Zemanski        Ben    M       57690        60190  2012
572      Zizzo        Sal    F       73125     81414.06  2012
573       Zusi     Graham  F-M      105000     138812.5  2012

[574 rows x 6 columns]
         Last Name First Name  Pos Base Salary Compensation  year
0         Abdallah      Aminu    M       46500        46500  2013
1           Acosta     Kellyn    M       46500        70500  2013
2         Adekugbe     Samuel    D

In [37]:
salary_dict['2012'].isna().sum()

Last Name       10
First Name      12
Pos             10
Base Salary      0
Compensation     0
year             0
dtype: int64

In [38]:
salary2012_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574 entries, 0 to 573
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Club          562 non-null    object
 1   Last Name     564 non-null    object
 2   First Name    562 non-null    object
 3   Pos           564 non-null    object
 4   Base Salary   574 non-null    object
 5   Compensation  574 non-null    object
 6   year          574 non-null    int64 
dtypes: int64(1), object(6)
memory usage: 31.5+ KB


In [39]:
salary_dict['2019'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714 entries, 0 to 713
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Last Name     714 non-null    object 
 1   First Name    705 non-null    object 
 2   Pos           713 non-null    object 
 3   Base Salary   714 non-null    float64
 4   Compensation  714 non-null    float64
 5   year          714 non-null    int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 33.6+ KB


In [40]:
salary_dict['2020'].drop(columns='Unnamed: 5', inplace=True)

In [41]:
salary_dict['2012'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2013'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2014'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2015'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2016'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2017'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2018'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2019'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']
salary_dict['2020'].columns = ['Last Name', 'First Name', 'Pos', 'Base Salary', 'Base Guaranteed Comp.', 'Year']

In [42]:
salary_df = pd.concat(salary_dict, ignore_index=True)  
salary_df

,Last Name,First Name,Pos,Base Salary,Base Guaranteed Comp.,Year
0,Adu,Freddy,M,400000,519000,2012
1,Agudelo,Juan,F,70000,100000,2012
2,Aiyegbusi,Olukorede,D,44100,44100,2012
3,Akpan,Andre,F,57200,57200,2012
4,Albright,Chris,D,55000,55000,2012
...,...,...,...,...,...,...
5856,NaN,Kaku,M,850000,850000,2020
5857,NaN,Robinho,M,198000,205500,2020
5858,NaN,Ilsinho,M,425000,476250,2020
5859,NaN,Judson,M,320000,325000,2020


In [43]:
salary_df['Pos'].value_counts()

Pos
M                   1771
D                   1643
F                   1117
GK                   643
M-F                  242
D-M                  149
Pos                   74
M-D                   48
F-M                   42
Playing Position      22
M/F                   15
M/D                    8
D/M                    6
F/M                    6
MF                     4
D/F                    4
Name: count, dtype: int64

In [44]:
player_salary_df = salary_df[salary_df['Pos'] != 'GK']
player_salary_df

,Last Name,First Name,Pos,Base Salary,Base Guaranteed Comp.,Year
0,Adu,Freddy,M,400000,519000,2012
1,Agudelo,Juan,F,70000,100000,2012
2,Aiyegbusi,Olukorede,D,44100,44100,2012
3,Akpan,Andre,F,57200,57200,2012
4,Albright,Chris,D,55000,55000,2012
...,...,...,...,...,...,...
5856,NaN,Kaku,M,850000,850000,2020
5857,NaN,Robinho,M,198000,205500,2020
5858,NaN,Ilsinho,M,425000,476250,2020
5859,NaN,Judson,M,320000,325000,2020


In [45]:
player_salary_df.dropna(subset=['First Name', 'Last Name'], how='all', inplace=True)

/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_17846/2152252057.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_salary_df.dropna(subset=['First Name', 'Last Name'], how='all', inplace=True)


In [46]:
player_salary_df.isna().sum()

Last Name                 5
First Name               28
Pos                       4
Base Salary               0
Base Guaranteed Comp.     0
Year                      0
dtype: int64

In [47]:
player_salary_df[player_salary_df['First Name'].isna()]

,Last Name,First Name,Pos,Base Salary,Base Guaranteed Comp.,Year
6,Alex,NaN,M,96000,105950,2012
258,Juninho,NaN,M,50000,65625,2012
583,Alex,NaN,M,110000,119950,2013
852,Juninho,NaN,M,290000,290000,2013
1177,Alex,NaN,M,123750,133700,2014
1336,Fabinho,NaN,D,96000,100500,2014
1441,Juninho,NaN,M,325000,325000,2014
1448,Kaka,NaN,M,6660000,7167500,2014
1773,Alex,NaN,M,142000,142000,2015
2025,Juninho,NaN,M,350000,350000,2015


In [48]:
for i, player in player_salary_df.iterrows():
    if pd.isnull(player['First Name']):
        player_salary_df.at[i, 'First Name'] = player_salary_df.at[i, 'Last Name']
    if pd.isnull(player['Last Name']):
        player_salary_df.at[i, 'Last Name'] = player_salary_df.at[i, 'First Name']

In [49]:
for i, player in player_salary_df.iterrows():
    if player_salary_df.at[i, 'First Name'] == player_salary_df.at[i, 'Last Name']:
        player_salary_df.at[i, 'Player'] = player_salary_df.at[i, 'Last Name']
    else:
        player_salary_df.at[i, 'Player'] = player_salary_df.at[i, 'First Name'] + " " +  player_salary_df.at[i, 'Last Name']

/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_17846/284201099.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_salary_df.at[i, 'Player'] = player_salary_df.at[i, 'First Name'] + " " +  player_salary_df.at[i, 'Last Name']
/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_17846/284201099.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Freddy Adu' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  player_salary_df.at[i, 'Player'] = player_salary_df.at[i, 'First Name'] + " " +  player_salary_df.at[i, 'Last Name']


In [50]:
player_salary_df

,Last Name,First Name,Pos,Base Salary,Base Guaranteed Comp.,Year,Player
0,Adu,Freddy,M,400000,519000,2012,Freddy Adu
1,Agudelo,Juan,F,70000,100000,2012,Juan Agudelo
2,Aiyegbusi,Olukorede,D,44100,44100,2012,Olukorede Aiyegbusi
3,Akpan,Andre,F,57200,57200,2012,Andre Akpan
4,Albright,Chris,D,55000,55000,2012,Chris Albright
...,...,...,...,...,...,...,...
5856,Kaku,Kaku,M,850000,850000,2020,Kaku
5857,Robinho,Robinho,M,198000,205500,2020,Robinho
5858,Ilsinho,Ilsinho,M,425000,476250,2020,Ilsinho
5859,Judson,Judson,M,320000,325000,2020,Judson


In [51]:
keeper_salary_df = salary_df[salary_df['Pos'] == 'GK']
keeper_salary_df

,Last Name,First Name,Pos,Base Salary,Base Guaranteed Comp.,Year
16,Angevine,Scott,GK,33756,33756,2012
42,Bendik,Joe,GK,44000,44000,2012
49,Bingham,David,GK,71000,115375,2012
74,Busch,Jon,GK,155004,166337.33,2012
75,Bush,Evan,GK,44000,44000,2012
...,...,...,...,...,...,...
5822,Westberg,Quentin,GK,240000,253333,2020
5828,Willis,Joe,GK,180000,187833,2020
5838,Yarbrough,William,GK,120000,120000,2020
5848,Zendejas,Adrian,GK,81375,81375,2020


In [52]:
keeper_salary_df['Player'] = keeper_salary_df['First Name'] + " " + keeper_salary_df['Last Name']
keeper_salary_df.drop(columns=['First Name', 'Last Name', 'Pos'], inplace=True)

/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_17846/208817401.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keeper_salary_df['Player'] = keeper_salary_df['First Name'] + " " + keeper_salary_df['Last Name']
/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_17846/208817401.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keeper_salary_df.drop(columns=['First Name', 'Last Name', 'Pos'], inplace=True)


In [53]:
def normalize_string(string):
    normalized = unidecode(string)
    return re.sub(r'[^A-Za-z0-9\s]+', '', string)
    return re.sub(r'[^\w\s]', '', normalized)

In [54]:
player_stats_df['Player'] = player_stats_df['Player'].apply(normalize_string)
player_salary_df['Player'] = player_salary_df['Player'].apply(normalize_string)
player_stats_df['Player'] = player_stats_df['Player'].str.lower()
player_salary_df['Player'] = player_salary_df['Player'].str.lower()

/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_17846/518822062.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_stats_df['Player'] = player_stats_df['Player'].apply(normalize_string)
/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_17846/518822062.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_salary_df['Player'] = player_salary_df['Player'].apply(normalize_string)
/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_17846/518822062.py:3: SettingWithCopyWarn

In [55]:
left_player_name = []
right_player_name = []
fuzz_player_name = []
for i, player in player_stats_df.iterrows():
    left_player = player['Player']
    left_player_name.append(left_player)
    closest_match = None
    highest_similarity = 0

    for _, right_player in player_salary_df.iterrows():
        right_player_1 = right_player['Player']
        similarity = fuzz.ratio(left_player, right_player_1)

        if similarity > highest_similarity:
            highest_similarity = similarity
            closest_match = right_player_1
            
    if highest_similarity < 80: 
        fuzz_player_name.append(left_player)
    else:
        fuzz_player_name.append(closest_match)
        player_stats_df.at[i, 'Player'] = closest_match

In [56]:
right_player_name = list(player_salary_df['Player'])

In [57]:
fuzz_df = pd.DataFrame({'Left Name': left_player_name,
                        'Fuzz Name': fuzz_player_name})
fuzz_df

,Left Name,Fuzz Name
0,chris wondolowski,chris wondolowski
1,kenny cooper,kenny cooper
2,alvaro saborio,alvaro saborio
3,robbie keane,robbie keane
4,thierry henry,thierry henry
...,...,...
5202,sebastian mendez,sebastian fernandez
5203,kelyn rowe,kelyn rowe
5204,siem de jong,siem de jong
5205,maxime chanot,maxime chanot


In [58]:
bad_fuzz = fuzz_df[fuzz_df['Left Name'] != fuzz_df['Fuzz Name']]

In [59]:
bad_fuzz

,Left Name,Fuzz Name
17,blas prez,blas perez
22,jaime castrilln,jaime castrillon
27,dwayne de rosario,dwayne derosario
41,gabriel gmez,gabriel gomez
53,sbastien le toux,sebastien le toux
...,...,...
5122,aljaz struna,aljaz kiki struna
5160,jeizon ramirez,jeizon ramirez chacon
5183,miguel ibarra,miguel ibarra
5187,matias vera,matias jara


In [60]:
fuzz.ratio('aljazstruna', 'aljazkikistruna')

84.61538461538461

In [61]:
keeper_stats_salaries_df = pd.merge(keeper_stats_df, keeper_salary_df, how='left',
                                    left_on=['Player', 'Year'], right_on=['Player', 'Year'])
keeper_stats_salaries_df                    

,Player,Club,POS,GP,GS,MINS,SHTS,SV,GA,GAA,...,W,L,T,ShO,W%,Sv%,Year,Season,Base Salary,Base Guaranteed Comp.
0,Jimmy Nielsen,KC,GK,34,34,3060,102,77,27,0.79,...,18,7,9,15,52.9,75.5,2012,reg,200000,220000
1,Donovan Ricketts,MTL,GK,33,33,2913,143,87,52,1.58,...,12,14,5,5,36.4,60.8,2012,reg,250000,275000
2,Andy Gruenebaum,CLB,GK,33,33,2920,166,124,41,1.24,...,13,12,7,8,39.4,74.7,2012,reg,77000,78666.67
3,Tally Hall,HOU,GK,33,33,2946,129,88,39,1.18,...,14,8,11,12,42.4,68.2,2012,reg,120000,129375
4,Jon Busch,SJ,GK,32,32,2835,123,77,40,1.25,...,17,5,9,7,53.1,62.6,2012,reg,155004,166337.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,Emmanuel Ochoa,SJ,GK,0,0,0,0,0,0,0.00,...,0,0,0,0,0.0,0.0,2020,reg,63547,73759
646,Drake Callender,MIA,GK,0,0,0,0,0,0,0.00,...,0,0,0,0,0.0,0.0,2020,reg,100000,100000
647,Elliot Panicco,NSH,GK,0,0,0,0,0,0,0.00,...,0,0,0,0,0.0,0.0,2020,reg,63547,63547
648,Eric Dick,SKC,GK,0,0,0,0,0,0,0.00,...,0,0,0,0,0.0,0.0,2020,reg,81375,85020


In [62]:
keeper_stats_salaries_df.isna().sum()

Player                    0
Club                      0
POS                       0
GP                        0
GS                        0
MINS                      0
SHTS                      0
SV                        0
GA                        0
GAA                       0
PKG/A                     0
W                         0
L                         0
T                         0
ShO                       0
W%                        0
Sv%                       0
Year                      0
Season                    0
Base Salary              73
Base Guaranteed Comp.    73
dtype: int64

In [63]:
player_stats_df[player_stats_df['SOG%'].isna()]

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,...,RdA,A/90min,FC,FS,OFF,YC,RC,SOG%,Year,Season
5328,roger torres,PHI,M,10,1,181,0,2,0,0,...,1,0.99,8,5,0,2,0,NaN,2012,reg
5329,adam johansson,SEA,D,21,21,1834,0,2,0,0,...,2,0.10,31,13,2,1,0,NaN,2012,reg
5348,terukazu tanaka,RSL,D,4,4,325,0,1,0,0,...,0,0.28,3,0,0,0,0,NaN,2012,reg
5349,michael harrington,KC,D,11,9,877,0,1,0,0,...,0,0.10,9,5,1,1,0,NaN,2012,reg
5396,steven kinney,CHI,D,0,0,0,0,0,0,0,...,0,0.00,0,0,0,0,0,NaN,2012,reg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15149,miguel berry,CLB,F,0,0,0,0,0,0,0,...,0,0.00,0,0,0,0,0,NaN,2020,reg
15150,nkosi burgess,DAL,D,0,0,0,0,0,0,0,...,0,0.00,0,0,0,0,0,NaN,2020,reg
15151,luke haakenson,NSH,M,0,0,0,0,0,0,0,...,0,0.00,0,0,0,0,0,NaN,2020,reg
15152,tony leone,LAFC,D,0,0,0,0,0,0,0,...,0,0.00,0,0,0,0,0,NaN,2020,reg


In [64]:
player_salary_df.drop(columns=['First Name', 'Last Name', 'Pos'], inplace=True)

/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_17846/3118917072.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_salary_df.drop(columns=['First Name', 'Last Name', 'Pos'], inplace=True)


In [65]:
player_salary_df = player_salary_df[player_salary_df['Player'] != 'First Name Last Name']
keeper_salary_df = keeper_salary_df[keeper_salary_df['Player'] != 'First Name Last Name']

In [66]:
player_stats_salaries_df = pd.merge(player_stats_df, player_salary_df, how='left', 
                                    left_on=['Player', 'Year'], right_on=['Player', 'Year'])
player_stats_salaries_df

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,...,FC,FS,OFF,YC,RC,SOG%,Year,Season,Base Salary,Base Guaranteed Comp.
0,chris wondolowski,SJ,F,32,31,2813,27,7,127,55,...,19,25,28,3,0,43.31,2012,reg,300000,300000
1,kenny cooper,NY,F,33,26,2505,18,3,84,46,...,36,28,20,3,0,54.76,2012,reg,275000,292500
2,alvaro saborio,RSL,F,31,26,2390,17,3,87,40,...,35,37,37,5,0,45.98,2012,reg,350000,405625
3,robbie keane,LA,F,28,28,2519,16,9,94,43,...,15,34,27,0,0,45.74,2012,reg,2917241,3417242.75
4,thierry henry,NY,F,25,24,2108,15,12,88,37,...,18,26,24,5,0,42.05,2012,reg,5000000,5600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5202,sebastian fernandez,ORL,M,19,11,1087,0,0,17,3,...,32,9,1,4,1,17.65,2020,reg,NaN,NaN
5203,kelyn rowe,NE,M,16,10,896,0,0,17,4,...,13,4,0,3,0,23.53,2020,reg,310610,310610
5204,siem de jong,CIN,M,15,8,793,0,0,18,5,...,7,8,1,0,0,27.78,2020,reg,1157556,1287556
5205,maxime chanot,NYC,D,20,20,1712,0,0,19,4,...,18,6,1,5,1,21.05,2020,reg,725000,725000


In [67]:
player_stats_salaries_df[player_stats_salaries_df['Player'] == 'felipe']

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,...,FC,FS,OFF,YC,RC,SOG%,Year,Season,Base Salary,Base Guaranteed Comp.
59,felipe,DC,M,30,30,2652,4,10,59,23,...,48,70,3,5,0,38.98,2012,reg,NaN,NaN
577,felipe,DC,M,32,28,2350,5,8,55,24,...,32,58,5,6,0,43.64,2013,reg,NaN,NaN
1168,felipe,DC,M,31,28,2456,3,6,60,17,...,43,51,4,5,0,28.33,2014,reg,NaN,NaN
1702,felipe,DC,M,34,34,2922,3,5,56,13,...,33,63,4,5,0,23.21,2015,reg,NaN,NaN
2213,felipe,DC,M,33,33,2898,5,7,54,22,...,51,66,0,6,1,40.74,2016,reg,NaN,NaN
2811,felipe,DC,M,33,33,2961,2,5,37,12,...,58,87,2,5,0,32.43,2017,reg,NaN,NaN
3464,felipe,DC,M,29,24,2203,1,7,26,8,...,29,56,1,4,0,30.77,2018,reg,NaN,NaN
4144,felipe,DC,M,27,22,1918,1,1,12,3,...,32,28,0,6,0,25.00,2019,reg,NaN,NaN
4840,felipe,DC,M,7,7,630,0,2,11,3,...,10,23,0,4,0,27.27,2020,reg,NaN,NaN


In [68]:
player_stats_salaries_df.isna().sum()

Player                      0
Club                        0
POS                         0
GP                          0
GS                          0
MINS                        0
G                           0
A                           0
SHTS                        0
SOG                         0
GWG                         0
PKG/A                       0
HmG                         0
RdG                         0
G/90min                     0
SC%                         0
GWA                         0
HmA                         0
RdA                         0
A/90min                     0
FC                          0
FS                          0
OFF                         0
YC                          0
RC                          0
SOG%                     1062
Year                        0
Season                      0
Base Salary               498
Base Guaranteed Comp.     498
dtype: int64

In [69]:
player_stats_salaries_df[player_stats_salaries_df['Base Salary'].isna()].head(30)

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,...,FC,FS,OFF,YC,RC,SOG%,Year,Season,Base Salary,Base Guaranteed Comp.
34,marco pappa,CHI,M,22,20,1615,6,5,69,21,...,28,32,3,5,0,30.43,2012,reg,NaN,NaN
53,sebastien le toux,NY,F,33,27,2522,5,2,34,16,...,17,24,18,0,0,47.06,2012,reg,NaN,NaN
59,felipe,DC,M,30,30,2652,4,10,59,23,...,48,70,3,5,0,38.98,2012,reg,NaN,NaN
62,boniek garcia,HOU,M,17,17,1530,4,6,30,12,...,19,31,4,0,0,40.00,2012,reg,NaN,NaN
72,bernardo corradi,MTL,F,11,7,619,4,1,13,7,...,14,14,12,3,0,53.85,2012,reg,NaN,NaN
128,paulo jr,RSL,F,22,12,848,2,2,26,12,...,13,7,12,0,0,46.15,2012,reg,NaN,NaN
146,jose erick correa,CHV,F,16,7,695,2,0,15,3,...,13,10,3,1,0,20.00,2012,reg,NaN,NaN
151,jerry bengtson,HON,F,13,10,942,2,0,22,8,...,15,17,9,1,0,36.36,2012,reg,NaN,NaN
186,jose villarreal,LA,M,10,0,122,1,1,3,2,...,7,2,0,1,0,66.67,2012,reg,NaN,NaN
192,martn bonjour,VAN,D,28,27,2396,1,1,8,2,...,43,16,2,7,0,25.00,2012,reg,NaN,NaN


In [70]:
player_stats_salaries_df[player_stats_salaries_df['Player'] == '']

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,...,FC,FS,OFF,YC,RC,SOG%,Year,Season,Base Salary,Base Guaranteed Comp.


In [71]:
for i, player in player_stats_salaries_df.iterrows():
    if player['Player'] in bad_fuzz['Fuzz Name'].values:
        player_stats_salaries_df.drop(i, inplace=True)

In [72]:
player_stats_salaries_df[player_stats_salaries_df['Base Salary'].isna()]

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,...,FC,FS,OFF,YC,RC,SOG%,Year,Season,Base Salary,Base Guaranteed Comp.
34,marco pappa,CHI,M,22,20,1615,6,5,69,21,...,28,32,3,5,0,30.43,2012,reg,NaN,NaN
59,felipe,DC,M,30,30,2652,4,10,59,23,...,48,70,3,5,0,38.98,2012,reg,NaN,NaN
62,boniek garcia,HOU,M,17,17,1530,4,6,30,12,...,19,31,4,0,0,40.00,2012,reg,NaN,NaN
72,bernardo corradi,MTL,F,11,7,619,4,1,13,7,...,14,14,12,3,0,53.85,2012,reg,NaN,NaN
128,paulo jr,RSL,F,22,12,848,2,2,26,12,...,13,7,12,0,0,46.15,2012,reg,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5116,wilfried zahibo,HOU,M,7,3,298,0,0,2,0,...,15,3,1,2,0,0.00,2020,reg,NaN,NaN
5136,sam nicholson,COL,F,2,2,174,0,0,3,0,...,2,3,2,0,0,0.00,2020,reg,NaN,NaN
5185,aleksandar katai,LA,M,2,2,117,0,0,8,0,...,5,2,0,2,0,0.00,2020,reg,NaN,NaN
5198,boniek garcia,HOU,M,18,7,819,0,0,12,1,...,12,9,0,5,0,8.33,2020,reg,NaN,NaN


In [73]:
player_stats_salaries_df.dropna(subset='Base Salary',inplace = True)

In [74]:
player_stats_salaries_df.isna().sum()

Player                     0
Club                       0
POS                        0
GP                         0
GS                         0
MINS                       0
G                          0
A                          0
SHTS                       0
SOG                        0
GWG                        0
PKG/A                      0
HmG                        0
RdG                        0
G/90min                    0
SC%                        0
GWA                        0
HmA                        0
RdA                        0
A/90min                    0
FC                         0
FS                         0
OFF                        0
YC                         0
RC                         0
SOG%                     816
Year                       0
Season                     0
Base Salary                0
Base Guaranteed Comp.      0
dtype: int64

In [79]:
keeper_stats_salaries_df.isna().sum()

Player                    0
Club                      0
POS                       0
GP                        0
GS                        0
MINS                      0
SHTS                      0
SV                        0
GA                        0
GAA                       0
PKG/A                     0
W                         0
L                         0
T                         0
ShO                       0
W%                        0
Sv%                       0
Year                      0
Season                    0
Base Salary              73
Base Guaranteed Comp.    73
dtype: int64

In [81]:
keeper_stats_salaries_df.dropna(subset='Base Salary', inplace=True)

In [83]:
player_stats_salaries_df.to_csv('./data/player_stats_salaries.csv')
keeper_stats_salaries_df.to_csv('./data/keeper_stats_salaries.csv')